In [2]:
import scipy.io

dataset = scipy.io.loadmat("Lorenz63Data.mat")['x']

In [3]:
dataset

array([[ 1.00000000e+00,  9.17927510e-01,  8.67924268e-01, ...,
        -6.77990147e+00, -7.12556121e+00, -7.49034267e+00],
       [ 0.00000000e+00,  2.66335809e-01,  5.11733691e-01, ...,
        -1.01378640e+01, -1.06794452e+01, -1.12300516e+01],
       [ 0.00000000e+00,  1.26355130e-03,  4.65403217e-03, ...,
         1.86069492e+01,  1.88253706e+01,  1.91138805e+01]])

In [7]:
import os
os.chdir("../../../")

In [10]:
from torchts.nn.models.ode import ODESolver
# from torchts.nn.model import TimeSeriesModel